In [35]:
!pip install catboost
import numpy as np
import pandas as pd
import sklearn
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import r2_score
import pickle

In [36]:
data = pd.read_csv('predict.csv')
data.dropna(subset=['c5'], inplace=True)
target = data['c5']
data.drop(['c5'], axis='columns', inplace=True)

In [37]:
nan_rate = {f'{column}' : data[column].isna().sum() / len(data[column]) for column in data.columns}
print(nan_rate)
drop_thresholder = 0.3
columns_to_drop = [k for k, v in nan_rate.items() if v > drop_thresholder]
data.drop(columns_to_drop, axis='columns', inplace=True)
data.head()

{'recency': 0.0, 'topic_rank': 0.0, 'diversity': 0.0, 'authors_mean_rank': 0.012526096033402923, 'authors_mean_hindex': 0.012526096033402923, 'authors_mean_gindex': 0.012526096033402923, 'authors_mean_sociality': 0.012526096033402923, 'authors_mean_pagerank': 1.0, 'authors_mean_productivity': 1.0, 'journal_pagerank': 1.0, 'journal_rank': 0.081419624217119, 'title_len': 0.0, 'abstract_len': 0.0, 'n_authors': 0.0, 'log_authors_mean_sociality': 0.012526096033402923}


,recency,topic_rank,diversity,authors_mean_rank,authors_mean_hindex,authors_mean_gindex,authors_mean_sociality,journal_rank,title_len,abstract_len,n_authors,log_authors_mean_sociality
0,16,1.0,-0.425436,500.0,1.0,1.0,6.0,101.0,66,1653,7,1.945910
1,8,7.0,-0.571967,134.0,1.0,1.0,6.0,24.0,59,1527,7,1.945910
2,15,19.0,-0.859644,235.0,1.0,1.0,5.0,49.0,169,1554,6,1.791759
3,6,4.0,-1.054999,1124.0,1.0,1.0,10.0,185.0,130,2121,11,2.397895
4,14,2.0,-0.476697,346.0,1.0,1.0,2.0,72.0,100,432,3,1.098612


In [38]:
imputer = IterativeImputer(random_state=42)
cols = data.columns
data = pd.DataFrame(imputer.fit_transform(data), columns=cols)
with open('imputer_dump.pkl', 'wb') as imp_dump:
    pickle.dump(imputer, imp_dump)

**Train**

In [5]:
train_part = 0.8
train_size = int(train_part * len(data))
X_train, X_val, Y_train, Y_val = train_test_split(data, target, train_size=train_size, random_state=42)



In [6]:
#def get_best_xgbreg_model(X_train, Y_train, model_parameters, scoring='r2', verbose=False):
    #reg = GridSearchCV(model, model_parameters, scoring=scoring, n_jobs=-1, verbose=verbose)
    #reg.fit(X_train, Y_train)
    #return reg.best_estimator_

In [165]:
#print(reg.best_estimator_.score(X_val, Y_val))

0.9252498109381974


In [30]:
model_params = {
    'learning_rate' : 0.16,
    'max_depth' : 3,
    'n_estimators' : 80, 
    'objective' : 'reg:squarederror', 
}

In [26]:
model = XGBRegressor(**model_params)
model.fit(X_train, Y_train,
          eval_set=[(X_val, Y_val)],
         )

model.score(X_val, Y_val)

[0]	validation_0-rmse:13.0056
[1]	validation_0-rmse:12.4335
[2]	validation_0-rmse:11.7659
[3]	validation_0-rmse:10.9175
[4]	validation_0-rmse:9.9664
[5]	validation_0-rmse:9.03892
[6]	validation_0-rmse:8.30704
[7]	validation_0-rmse:7.55188
[8]	validation_0-rmse:7.24996
[9]	validation_0-rmse:6.7225
[10]	validation_0-rmse:6.17761
[11]	validation_0-rmse:6.1143
[12]	validation_0-rmse:5.64964
[13]	validation_0-rmse:5.52612
[14]	validation_0-rmse:5.23322
[15]	validation_0-rmse:4.96746
[16]	validation_0-rmse:4.68541
[17]	validation_0-rmse:4.48229
[18]	validation_0-rmse:4.42872
[19]	validation_0-rmse:4.33312
[20]	validation_0-rmse:4.29421
[21]	validation_0-rmse:4.15052
[22]	validation_0-rmse:4.10705
[23]	validation_0-rmse:4.05143
[24]	validation_0-rmse:3.99292
[25]	validation_0-rmse:3.99096
[26]	validation_0-rmse:3.97541
[27]	validation_0-rmse:3.88744
[28]	validation_0-rmse:3.85695
[29]	validation_0-rmse:3.77581
[30]	validation_0-rmse:3.72263
[31]	validation_0-rmse:3.73786
[32]	validation_0-rms

0.9252498109381974